In [1]:
import os
import pickle
import re
from glob import glob
from ipywidgets import FloatProgress
from IPython.display import display
import matplotlib.pyplot as plt
import datetime

flag_dscrpt = {
        1 : 'NOT A DISTB MECH',
        2 : 'NO STATES DEFINED',
        3 : 'STATES HAS FROM 0 TO 1',
        11 : 'NO OUTPUT FROM HHAnalyse',
        19 : '#STATES != #OUTPUT FILES',
        30 : 'UNPARSE Unknown',
        31 : 'UNPARSE TextXSyntaxError',
        32 : 'UNPARSE TextXSemanticError',
        33 : 'UNPARSE AttributeError'
}

def get_flag_counts(mega_dict, flag_typs=[]):
    count = 0
    for channel in mega_dict:
        if 'red_flag' in mega_dict[channel]:
            if not flag_typs:
                count += 1
            else:
                for flag_typ in flag_typs:
                    if flag_typ in mega_dict[channel]['red_flag']:
                        count += 1 
    return count

def get_flag_channel(mega_dict, flag_typs=[]):
    channel_list = []
    for channel in mega_dict:
        if 'red_flag' in mega_dict[channel]:
            if not flag_typs:
                channel_list.append(channel)
            else:
                for flag_typ in flag_typs:
                    if flag_typ in mega_dict[channel]['red_flag']:
                        channel_list.append(channel)    
    return set(channel_list)

def show_progress(description, value):
    f = FloatProgress(min=0, max=100, description=description + ' : ')
    f.value = value
    return f

def print_progress(mega_dict, flg_typ):
    f_channels = get_flag_counts(mega_dict, [flg_typ])
    percent_f_cover = float(f_channels*100. / brkn_channel_count)
    print('Total number of broken channels : ', brkn_channel_count)
    print('Total number of ', flag_dscrpt[flg_typ], ' : ', f_channels)
    print('Percent of the failed : ', percent_f_cover)
    return show_progress(flag_dscrpt[flg_typ], percent_f_cover)

def fail_percent(mega_dict, flag_list):
    counts = []
    for flg in flag_list:
        counts.append(len(get_flag_channel(mega_dict, [flg])))
    counts = [(100.0 * ii) / len(mega_dict) for ii in counts]
    return counts

def make_plot():
    now = datetime.datetime.now()
    pos = [ii + 0.5 for ii in range(len(flag_dscrpt)+1)]
    values = [100.-percent_cover]
    colors = ['g']
    labels = ['ALL PASS']
    for key, value in flag_dscrpt.items():
        values.append(fail_percent(mega_dict, [key])[0])
        colors.append('r')
        labels.append(value)
    plt.figure(figsize=(10,10))
    bar_list = plt.barh(pos, values)
    for ii, bar in enumerate(bar_list):
        bar.set_color(colors[ii])
    plt.yticks(pos, labels)
    ax = plt.gca()
    ax.set_xlim(0,100)
    ax.set_xlabel('Percentage')
    plt.title(sub_channel + ' ' + now.strftime("%Y-%m-%d %H:%M"))
    #plt.savefig(sub_channel+'_coverage.png', dpi=150, bbox_inches='tight')
    plt.show()

In [2]:
sub_channel = 'icg-channels-IH'
with open(sub_channel+'.pkl', 'rb') as handle:
    mega_dict = pickle.load(handle)

brkn_channel_count = get_flag_counts(mega_dict)
percent_cover = float(brkn_channel_count)/len(mega_dict)*100.

In [3]:
make_plot()

In [4]:
print('Total number of channels in, ', sub_channel, ' : ', len(mega_dict))
print('Total number of broken channels : ', brkn_channel_count)
print('Number of channels that are OKAY so far: ', len(mega_dict)-brkn_channel_count)
print('Percent of pass : ', percent_cover)
display(show_progress('Pass coverage', percent_cover))

Total number of channels in,  icg-channels-IH  :  243
Total number of broken channels :  237
Number of channels that are OKAY so far:  6
Percent of pass :  97.53086419753086


A Jupyter Widget

In [5]:
display(print_progress(mega_dict, 31))
print(get_flag_channel(mega_dict, [31]))

Total number of broken channels :  237
Total number of  UNPARSE TextXSyntaxError  :  25
Percent of the failed :  10.548523206751055


A Jupyter Widget

{'105385_hcn12_gp.mod', '135903_iholmw.mod', '135903_ihpyrkop.mod', '185858_ihlts.mod', '122329_hcn12_gp.mod', '139421_iholmkop.mod', '151282_iholmw.mod', '64229_arhRT03_v2.mod', '97860_hcn_siegelbaum.mod', '135903_iholmkop.mod', '98005_hcn12_ch.mod', '105385_hcn2_gp.mod', '98005_hcn2_ch.mod', '182134_iholmkop.mod', '122329_hcn2_gp.mod', '182134_ihpyrkop.mod', '135902_iholmkop.mod', '139421_ihpyrkop.mod', '182134_iholmw.mod', '64229_arhRT03.mod', '135902_ihpyrkop.mod', '155735_ihpkj_cn.mod', '19176_hcn2.mod', '135902_iholmw.mod', '139421_iholmw.mod'}
